In [1]:
from ae_models import ae_model
from keras.models import Model
import keras
from keras.layers import Input,Dense,Dropout
from keras import backend as K
import numpy as np
import pandas as pd
import random

Using TensorFlow backend.


In [7]:
class_num=10
speed1=1772
speed2=800
model_name='contractive'
trainx=pd.read_csv(f'../data/CRWU/x_{speed1}_{class_num}_f.csv')
trainy=pd.read_csv(f'../data/CRWU/y_{speed1}_{class_num}.csv')
#x_test=pd.read_csv(f'../data/CRWU/x_{speed2}_{class_num}_f.csv').values
#y_test=pd.read_csv(f'../data/CRWU/y_{speed2}_{class_num}.csv').values
#trainx=pd.read_csv(f'../data/JN/x_{speed1}_2_f.csv').iloc[:4000]
#trainy=pd.read_csv(f'../data/JN/y_{speed1}_2.csv').iloc[:4000]
#x_test=pd.read_csv(f'../data/JN/x_{speed2}_2_f.csv').iloc[:4000].values
#y_test=pd.read_csv(f'../data/JN/y_{speed2}_2.csv').iloc[:4000].values
index =list(trainx.index)
random.Random(0).shuffle(index)
num=int(len(index)/10*5)
x_train,y_train=trainx.iloc[index[:num],:].values,trainy.iloc[index[:num],:].values
x_test,y_test=trainx.iloc[index[num:],:].values,trainy.iloc[index[num:],:].values
#data_X=np.vstack([x_train,x_test])
def trans(data):
    nm=np.zeros(data.shape)
    for i in range(len(data)):
        nm[i,:]=(data[i,:])/np.max(data[i,:])
    return nm
x_train=trans(x_train)
x_test=trans(x_test)
data_X=np.vstack([x_train,x_test])

### 稀疏自编码

In [8]:
K.clear_session()

input_dim=1024
input_img=Input(shape=(input_dim,))
ae=ae_model(input_dim,[400,200,64],0.1,0.01,0)
ae_out=ae.auto_encode(input_img)
ae_mode=Model(inputs=input_img,outputs=ae_out)

lam=1e-3
def contractive_loss(y_pred, y_true):
    mse = K.mean(K.square(y_true - y_pred), axis=1)
    contractive=0
    for ename in ['eh1','eh2']:
        W = K.variable(value=ae_mode.get_layer(ename).get_weights()[0])  # N x N_hidden
        W = K.transpose(W)  # N_hidden x N
        h = ae_mode.get_layer(ename).output
        dh = h * (1 - h)  # N_batch x N_hidden
        contractive+= lam * K.sum(dh**2 * K.sum(W**2, axis=1), axis=1)
    return mse + contractive

#分类
encode_out=ae.encode(input_img)
LR=Dense(class_num,activation='softmax',name='LR')(encode_out)
ae_mode.compile(optimizer='adam',loss=contractive_loss)
hist=ae_mode.fit(data_X,data_X,epochs=8,batch_size=128,shuffle=True)

Epoch 1/8
4606/4606 [==============================] - 1s 202us/step - loss: 4.9552
Epoch 2/8
4606/4606 [==============================] - 0s 105us/step - loss: 0.9875
Epoch 3/8
4606/4606 [==============================] - 0s 104us/step - loss: 0.2298
Epoch 4/8
4606/4606 [==============================] - 0s 102us/step - loss: 0.0693
Epoch 5/8
4606/4606 [==============================] - 0s 109us/step - loss: 0.0259
Epoch 6/8
4606/4606 [==============================] - 0s 105us/step - loss: 0.0132
Epoch 7/8
4606/4606 [==============================] - 0s 104us/step - loss: 0.0097
Epoch 8/8
4606/4606 [==============================] - 0s 99us/step - loss: 0.0080


In [9]:
#定义损失函数2（默认为categorical_crossentropy）
def contractive_loss2(y_pred, y_true):
    cost = K.categorical_crossentropy(y_true,y_pred)
    contractive=0
    for ename in ['eh0','eh1','LR']:
        W = K.variable(value=encoder_mode.get_layer(ename).get_weights()[0])  # N x N_hidden
        W = K.transpose(W)  # N_hidden x N
        h = encoder_mode.get_layer(ename).output
        dh = h * (1 - h)  # N_batch x N_hidden
        contractive+= lam * K.sum(dh**2 * K.sum(W**2, axis=1), axis=1)
    return cost + contractive
encoder_mode=Model(input_img,LR)
encoder_mode.compile(optimizer='adam',loss=contractive_loss2,metrics=['categorical_accuracy'])
encoder_mode.fit(x_train,y_train,epochs=50,batch_size=128,shuffle=True)
score=encoder_mode.evaluate(x_test,y_test)
print(score)

Epoch 1/50
2303/2303 [==============================] - 1s 244us/step - loss: 18.1939 - categorical_accuracy: 0.5649
Epoch 2/50
2303/2303 [==============================] - 0s 73us/step - loss: 9.3999 - categorical_accuracy: 0.8215
Epoch 3/50
2303/2303 [==============================] - 0s 73us/step - loss: 6.3781 - categorical_accuracy: 0.8997
Epoch 4/50
2303/2303 [==============================] - 0s 84us/step - loss: 4.9623 - categorical_accuracy: 0.9201
Epoch 5/50
2303/2303 [==============================] - 0s 90us/step - loss: 4.2837 - categorical_accuracy: 0.9201
Epoch 6/50
2303/2303 [==============================] - 0s 83us/step - loss: 3.8235 - categorical_accuracy: 0.9201
Epoch 7/50
2303/2303 [==============================] - 0s 75us/step - loss: 3.4952 - categorical_accuracy: 0.9201
Epoch 8/50
2303/2303 [==============================] - 0s 85us/step - loss: 3.0231 - categorical_accuracy: 0.9449
Epoch 9/50
2303/2303 [==============================] - 0s 92us/step - loss: 1

In [5]:
probs=encoder_mode.predict(x_test)
def save_data(data,name):
    sdata=pd.DataFrame(data,columns=list(range(data.shape[1])))
    sdata.to_csv(f'./prob/{name}.csv',encoding='utf-8',index=None)
save_data(probs,f'{speed1}_{speed2}_{model_name}')
encoder_mode.save(f'./model/model_{speed1}_{speed2}_{model_name}_m.h5')
encoder_mode.save_weights(f'./model/model_{speed1}_{speed2}_{model_name}_w.h5')
with open('predict_prob.txt','a+') as f:
    f.write(f'{model_name}_{speed1}_{speed2}'+'----'+str(score)+'\n')